# MONTE CARLO NÃO-SEQUENCIAL

Trabalho de Confiabilidade de Sistemas Elétricos de Potência

Alunos: 

Gabriel Halfeld

Vinícius Albuquerque Cabral

Data de entrega: 17/12/2024

## IMPORTAÇÃO DE BIBLIOTECAS

Nesta seção, são importadas as bibliotecas necessárias para a realização das análises de confiabilidade. As principais bibliotecas utilizadas são:

- `Pyomo`: Para modelagem e solução de problemas de otimização.
- `pandas` (`pd`): Para manipulação e análise de dados tabulares.
- `numpy` (`np`): Para operações matemáticas e computacionais.
- `data_processing`: Um módulo personalizado para carregar os dados do sistema.
- `calcula_fpo`: Um módulo personalizado para calcular os fluxos de potência.

As bibliotecas externas são carregadas utilizando comandos `import`, enquanto os módulos customizados (localizados no diretório do projeto) são carregados diretamente.

In [6]:
import pyomo as pyo
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo
import time

## LEITURA E PROCESSAMENTO DOS DADOS
Nesta seção, os dados necessários para a análise de confiabilidade são carregados e processados. A estrutura geral envolve:

- **`D_GEN`**: Dados relacionados aos geradores, incluindo taxas de falha, tempo de reparo e disponibilidade.
- **`D_LIN`**: Dados das linhas de transmissão, considerando falhas por milha, comprimento das linhas e tempo de reparo.
- **`D_LOAD`**: Informações sobre as cargas no sistema.

Os dados carregados são normalizados para uso em análises temporais e de confiabilidade, incluindo a conversão de tempos e taxas para a escala de horas.



In [2]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

In [3]:
#Deixando os dados em horas e nos formatos conhecidos
#Dados de Geração
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

#Dados de Linha
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760 # 8760 horas em um ano
D_LIN['MTTF'] = 1 / D_LIN['Lambda'] #Tempo médio entre falhas
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

#Numero de componentes
Ng = len(D_GEN)
Nl = len(D_LIN)
Nc = len(D_LOAD)


## ALGORITMO PRINCIPAL SMC NÃO-SEQUENCIAL
O algoritmo de Monte Carlo Não-Sequencial (SMC-NS) é utilizado para simular o comportamento do sistema elétrico sem dependência temporal, levando em consideração falhas e reparos simultâneos de componentes, como geradores e linhas de transmissão. O objetivo é calcular índices de confiabilidade, como LOLP (Loss of Load Probability), EPNS (Expected Power Not Supplied), EENS (Energy Not Supplied), LOLE (Loss of Load Expectation), LOLD (Loss of Load Duration) e LOLF (Loss of Load Frequency).

1. Variáveis de Controle para o Loop de Monte Carlo:
NS: Contador de simulações realizadas.
NSmax: Limite máximo de simulações (20000).
beta: Convergência do LOLP (Loss of Load Probability).
beta_tol: Tolerância do LOLP.
beta_EPNS: Convergência do EPNS (Expected Power Not Supplied).
beta_EPNS_tol: Tolerância do EPNS.
São também inicializadas variáveis para acumular as somas de LOLP, EPNS e suas variâncias, a fim de calcular a convergência.

2. Loop de Monte Carlo:
O loop principal do SMC-NS é executado até que o número máximo de simulações (NSmax) seja atingido ou até que o LOLP e o EPNS atinjam a convergência desejada.
- Geração de Sorteios Aleatórios: Para cada simulação, sorteios aleatórios são realizados para determinar o estado de falha de cada gerador e linha de transmissão:
    - Estado dos Geradores: Para cada gerador, um sorteio é feito com base na taxa de falha do gerador (D_GEN['U'][i]).
    - Estado das Linhas: Para cada linha de transmissão, um sorteio é feito com base na taxa de falha da linha (D_LIN['U'][i]).
- Verificação de Perda de Carga: A função calcular_fpo é chamada para verificar se há perda de carga, isto é, se o corte (corte) é maior que zero.
- Atualização do LOLP e EPNS: Se houver perda de carga, o LOLP e o EPNS são atualizados com base nas perdas de carga.
- Sorteio de Tempos de Reparo: Para cada componente (geradores e linhas) que falhou, o tempo de reparo é sorteado a partir de uma distribuição exponencial, com taxa '1/mu = MTTR':
    - O algoritmo escolhe o menor tempo de reparo entre os geradores e as linhas e realiza o reparo do componente correspondente. Após o reparo, o estado do componente é atualizado para "operacional" (1).
    - Após cada reparo, o corte é recalculado para verificar se a perda de carga foi eliminada.
    - O processo de sorteio de tempos de reparo é repetido até que não haja mais perda de carga e obtivermos o tempo de reparo da falha.



In [4]:
# Variáveis de controle para o loop de Monte Carlo
NS = 0                     # Contador de simulações
NSmax = 20000              # Limite de simulações

beta = 10                  # Convergência de LOLP
beta_tol = 0.01            # Tolerância de LOLP

beta_EPNS = 10             # Convergência de EPNS
beta_EPNS_tol = 0.02       # Tolerância de EPNS

# Variáveis de acumulação para LOLP e EPNS
SLOLP, SLOLP2 = 0, 0       # Somas de LOLP e quadrado
SEPNS, SEPNS2 = 0, 0       # Somas de EPNS e quadrado

tempo_falha_total = []

# Loop de Monte Carlo
while (beta >= beta_tol or beta_EPNS >= beta_EPNS_tol) and NS <= NSmax:
    NS += 1
    # Geração de sorteios aleatórios para geradores e linhas
    sorteio_ger = np.random.rand(Ng)
    sorteio_linhas =  np.random.rand(Nl)
    estado_ger = [1 if sorteio_ger[i] > D_GEN['U'][i] else 0 for i in range(Ng)]
    estado_linhas = [1 if sorteio_linhas[i] > D_LIN['U'][i] else 0 for i in range(Nl)]

    # Verificação de perda de carga: Se corte > 0, há perda de carga
    corte = calcular_fpo(estado_ger, estado_linhas, False)

    if corte > 0:
        #Atualização das somas do LOLP e EPNS
        SLOLP += 1
        SLOLP2 += 1**2
        SEPNS += corte
        SEPNS2 += corte**2

        ger_falhos = [i for i in range(Ng) if estado_ger[i] == 0]
        lin_falhos = [i for i in range(Nl) if estado_linhas[i] == 0]

        # Sorteando o tempo de reparo para cada componente falho (exponencial com taxa mu)
        tempos_reparo_geradores = {i: np.random.exponential(1/D_GEN['mu'][i]) for i in ger_falhos}
        tempos_reparo_linhas = {i: np.random.exponential(1/D_LIN['mu'][i]) for i in lin_falhos}
        tempo_falha = 0
        
        while corte > 0:
            menor_tempo_ger = np.inf
            menor_tempo_lin = np.inf
            # Definindo o menor tempo de reparo e o componente correspondente
            if tempos_reparo_geradores: # Verifica se o dicionário não está vazio
                menor_tempo_ger = min(tempos_reparo_geradores.values())
                componente_menor_tempo_ger = min(tempos_reparo_geradores, key=tempos_reparo_geradores.get)
            if tempos_reparo_linhas: # Verifica se o dicionário não está vazio
                menor_tempo_lin = min(tempos_reparo_linhas.values())
                componente_menor_tempo_lin = min(tempos_reparo_linhas, key=tempos_reparo_linhas.get)

            #Escolhendo o menor tempo de reparo:
            if menor_tempo_ger < menor_tempo_lin:
                tempo_falha = menor_tempo_ger
                # Atualizar o estado do gerador e remover o tempo de reparo do dicionário
                estado_ger[componente_menor_tempo_ger] = 1
                del tempos_reparo_geradores[componente_menor_tempo_ger]
            else:
                tempo_falha = menor_tempo_lin
                # Atualizar o estado da linha e remover o tempo de reparo do dicionário
                estado_linhas[componente_menor_tempo_lin] = 1
                del tempos_reparo_linhas[componente_menor_tempo_lin]
            #Calcular o corte
            corte = calcular_fpo(estado_ger, estado_linhas, False)
        tempo_falha_total.append(tempo_falha)
    
    # Cálculo de convergência após 100 iterações
    if NS > 100:
        # Cálculo de LOLP e beta
        ELOLP = SLOLP / NS
        varLOLP = (SLOLP2 - NS * ELOLP**2) / (NS - 1)
        varELOLP = varLOLP / NS
        beta = np.sqrt(varELOLP) / ELOLP if ELOLP != 0 else 0

        # Cálculo de EPNS e beta_EPNS
        EPNS = SEPNS / NS
        varEPNS = (SEPNS2 - NS * EPNS**2) / (NS - 1)
        varEPNS = varEPNS / NS
        beta_EPNS = np.sqrt(varEPNS) / EPNS if EPNS != 0 else 0

#Obtenção do EPNS:
EPNS_MC = EPNS
#Obtenção do LOLP:
LOLP_MC = ELOLP
#Cálculo do EENS:
EENS_MC = EPNS * 8760
#Cálculo do LOLE:
LOLE_MC = 8760 * LOLP_MC
#Cálculo do LOLD:
LOLD_MC = sum(tempo_falha_total)/len(tempo_falha_total)
#Cálculo do LOLF:
LOLF_MC = LOLE_MC / LOLD_MC

## RESULTADOS
Nesta seção, os resultados obtidos serão apresentados:
- **LOLP (Loss of Load Probability)**
- **LOLE (Loss of Load Expectation)**
- **LOLF (Loss of Load Frequency)**
- **EENS (Expected Energy Not Supplied)**
- **EPNS (Expected Power Not Supplied)**
- **LOLD (Loss of Load Duration)**


In [5]:
# Print dos Resultados:
print(f"Expected Energy Not Supplied (EENS): {EENS_MC/1000:.2f} GWh/ano")
print(f"Expected Power Not Supplied (EPNS): {EPNS_MC:.2f} MW")
print(f"Loss of Load Duration (LOLD): {LOLD_MC:.2f} horas/eventos de corte")
print(f"Loss of Load Expectation (LOLE): {LOLE_MC:.2f} horas/ano")
print(f"Loss of Load Probability (LOLP): {LOLP_MC*100:.2f}  %")
print(f"Loss of Load Frequency (LOLF): {LOLF_MC:.2f} eventos de corte/ano")
print('Tempo de Execução: 19m 9.8s')

Expected Energy Not Supplied (EENS): 56.43 GWh/ano
Expected Power Not Supplied (EPNS): 6.44 MW
Loss of Load Duration (LOLD): 33.64 horas/eventos de corte
Loss of Load Expectation (LOLE): 800.19 horas/ano
Loss of Load Probability (LOLP): 9.13  %
Loss of Load Frequency (LOLF): 23.79 eventos de corte/ano
Tempo de Execução: 19m 9.8s


## Outra forma de definir o LOLD e LOLF

Uma outra abordagem para definir o LOLD é utilizando os valores determinísticos do MTTR de todos os componentes falhos e escolher o maior valor, ou seja, o maior tempo de reparo. Dessa forma, fazemos uma aproximação do tempo de reparo para o reparo de todos os componentes falhos, diferentemente do que fizemos anteriormente, em que reparávamos um componente por vez e verificávamos a perda de carga em cada reparo. Além disso, anteriormente faziamos uma amostragem do tempo de reparo com uma distribuição exponencial, enquanto que agora nos baseamos com os valores determinísticos do MTTR.

Essa abordagem reduz o tempo computacional, mas superestima o LOLD e, em compensação, subestima o LOLF.

In [33]:
tic = time.time()

Ncenarios = 1000000

#  0        1         2                3       4      5        6       7       8         9               10
# LOLP | ELOLP | SOMATORIO ELOLP^2 | EPNS | EEPNS | LOLE  | ELOLE | EENS  | EEENS | SOMATORIO EENS^2 | mu_estado
MATRIZ_CENARIOS = np.zeros((Ncenarios,10))

# Vai armazenar os valores de cada índice Beta
BETA = np.zeros((Ncenarios,))

beta = 1
icen = -1
conta_indisp = 0

while beta > 5/100 and icen < Ncenarios-1:

    icen += 1
    
    # Gera o estado
    sorteio_ger = np.random.rand(Ng)
    sorteio_lin = np.random.rand(Nl)

    GER_ON = np.array(D_GEN['U']) <= sorteio_ger
    LIN_ON = np.array(D_LIN['U']) <= sorteio_lin

    GER_ON = GER_ON.astype(int)
    LIN_ON = LIN_ON.astype(int)

    corte = calcular_fpo(GER_ON, LIN_ON, False)
    
    tempo_reparo_sistema = np.max([np.max((1-GER_ON)*D_GEN['Tempo de reparo(h)']), np.max((1-LIN_ON)*D_LIN['Tempo de Reparo'])])

    if corte > 0:

        MATRIZ_CENARIOS[icen,0] = 1
        MATRIZ_CENARIOS[icen,5] = tempo_reparo_sistema

    MATRIZ_CENARIOS[icen,3] = corte

    # Construção do índice ELOLP e EEPNS
    if icen == 0:
        MATRIZ_CENARIOS[icen,1] = MATRIZ_CENARIOS[icen,0]
        MATRIZ_CENARIOS[icen,4] = MATRIZ_CENARIOS[icen,3]
    else:
        MATRIZ_CENARIOS[icen,1] = np.sum(MATRIZ_CENARIOS[:icen+1,0])/(icen+1)
        MATRIZ_CENARIOS[icen,4] = np.sum(MATRIZ_CENARIOS[:icen+1,3])/(icen+1)
    
    # Construção do índice ELOLP^2
    if icen == 0:
        MATRIZ_CENARIOS[icen,2] = MATRIZ_CENARIOS[icen,0]
    else:
        MATRIZ_CENARIOS[icen,2] = np.sum(MATRIZ_CENARIOS[:icen+1,0]**2)

    if icen > 0:

        # Variância da função
        var_fun = (MATRIZ_CENARIOS[icen,2] - (icen+1)*MATRIZ_CENARIOS[icen,1]**2)/icen

        # Variância do valor esperado
        var_esp = var_fun/(icen+1)

        desvio_padrao_valor_esperado = np.sqrt(var_esp)

        beta = desvio_padrao_valor_esperado/(MATRIZ_CENARIOS[icen,1]+0.0000000001)

        if beta == 0:

            beta = 1

        if icen % 1000 == 0:

            print(icen,beta)

        BETA[icen] = beta

1000 0.09822899851704987
2000 0.07112196089877565
3000 0.056802920352788736


In [34]:
MATRIZ_CENARIOS = MATRIZ_CENARIOS[:icen+1,:]

# Cálculo do LOLP do sistema
LOLP_MC = MATRIZ_CENARIOS[-1,1]
print('LOLP - LOSS OF LOAD PROBABILITY')
print('Probabilidade de ocorrência de perda de carga')
print('O LOLP é calculado através da divisão da quantidade de cenários em que há perda de carga pela quantidade total de cenários:')
print('LOLP:',np.round(LOLP_MC*100,2),'%')
print('----------------------------------------------')

# Cálculo do LOLE do sistema
LOLE_MC = LOLP_MC*8760
print('LOLE - LOSS OF LOAD EXPECTATION')
print('É o tempo médio esperado em que não haverá atendimento da carga do sistema.')
print('O LOLE é calculado através da multiplicação :')
print('LOLE:',np.round(LOLE_MC,2),'h')
print('----------------------------------------------')

# Cálculo do EPNS do sistema
EPNS_MC = np.sum(MATRIZ_CENARIOS[:,4])/(icen+1)
print('EPNS - EXPECTED POWER NOT SUPPLIED')
print('Valor experado da carga não atendida, em MW.')
print('EPNS:',np.round(EPNS_MC,2),'MW')
print('----------------------------------------------')

# Cálculo do EENS do sistema
EENS_MC = EPNS_MC*8760
print('EENS - EXPECTED ENERGY NOT SUPPLIED')
print('Valor experado da energia não atendida, em MWh.')
print('EENS:',np.round(EENS_MC,2),'MWh')
print('----------------------------------------------')

# Cálculo do LOLD do sistema
LOLD_MC = np.sum(MATRIZ_CENARIOS[:,5])/sum(MATRIZ_CENARIOS[:,0])
print('LOLD - LOSS OF LOAD DURATION')
print('Duração média da perda de carga.')
print('Pela definição, o LOLD é a razão entre o LOLP e o LOLF.')
print('LOLD:',np.round(LOLD_MC,2),'h')
print('----------------------------------------------')

# Cálculo do LOLF do sistema
LOLF_MC = LOLP_MC/LOLD_MC*8760
print('LOLF - LOSS OF LOAD FREQUENCY')
print('É a frequência média de perda de carga definida como falhas/período')
print('O LOLF é calculado através da divisão da soma das taxas de reparos dos cenários em que há perda de carga pela quantidade de cenários:')
print('LOLF:',np.round(LOLF_MC,2),'falhas/ano')
print('----------------------------------------------')

LOLP - LOSS OF LOAD PROBABILITY
Probabilidade de ocorrência de perda de carga
O LOLP é calculado através da divisão da quantidade de cenários em que há perda de carga pela quantidade total de cenários:
LOLP: 9.57 %
----------------------------------------------
LOLE - LOSS OF LOAD EXPECTATION
É o tempo médio esperado em que não haverá atendimento da carga do sistema.
O LOLE é calculado através da multiplicação :
LOLE: 838.48 h
----------------------------------------------
EPNS - EXPECTED POWER NOT SUPPLIED
Valor experado da carga não atendida, em MW.
EPNS: 6.7 MW
----------------------------------------------
EENS - EXPECTED ENERGY NOT SUPPLIED
Valor experado da energia não atendida, em MWh.
EENS: 58690.7 MWh
----------------------------------------------
LOLD - LOSS OF LOAD DURATION
Duração média da perda de carga.
Pela definição, o LOLD é a razão entre o LOLP e o LOLF.
LOLD: 85.69 h
----------------------------------------------
LOLF - LOSS OF LOAD FREQUENCY
É a frequência média de 